## $$\text{Algoritmo XGBoost}$$

In [98]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

dataset = pd.read_csv('./data/Churn_Modelling.csv')
X = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values

X[:3]

array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 'Female', 42, 8, 159660.8, 3, 1, 0, 113931.57]],
      dtype=object)

### Variables categóricas

In [99]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer

# label1 = LabelEncoder()
# X[:,1] = label1.fit_transform(X[:,1])
label2 = LabelEncoder()
X[:,2] = label2.fit_transform(X[:,2])
onehotencoder = make_column_transformer((OneHotEncoder(), [1]), remainder = "passthrough") #(OneHotEncoder(), [1,2])
X = onehotencoder.fit_transform(X)
X = X[:,1:]
X[:3]

array([[0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57]],
      dtype=object)

### Dividir datos

In [102]:
from sklearn.model_selection import train_test_split 

xtrain, xtest, ytrain, ytest = train_test_split(X,y,test_size=.2,random_state=0)

### Escalado

In [103]:
from sklearn.preprocessing import StandardScaler 

scx= StandardScaler()
xtrain = scx.fit_transform(xtrain)
xtest = scx.transform(xtest)

### xgboost

In [105]:
from xgboost import XGBClassifier 

classifier = XGBClassifier()
classifier.fit(xtrain,ytrain)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

### predicción y matriz de confusión

In [132]:
ypred = classifier.predict(xtest)
# ypred = (ypred > 0.5)

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ytest,ypred)
cm

array([[1491,  104],
       [ 187,  218]])

In [134]:
print(f'verdaderos positivos: {cm[0,0]}')
print(f'falsos positivos: {cm[0,1]}')
print(f'falsos negativos: {cm[1,0]}')
print(f'verdaderos negativos: {cm[1,1]}')

verdaderos positivos: 1491
falsos positivos: 104
falsos negativos: 187
verdaderos negativos: 218


### validación cruzada

In [130]:
from sklearn.model_selection import cross_val_score 

accuracies = cross_val_score(estimator=classifier, X=xtrain, y=ytrain, cv=10)
print(accuracies)
print(f'mean: {accuracies.mean()}')
print(f'std: {accuracies.std()}')

[0.865   0.84    0.87125 0.8475  0.84625 0.84375 0.865   0.84625 0.83875
 0.865  ]
mean: 0.852875
std: 0.01159269705461158
